In [ ]:
#import sys
#!{sys.executable} -m pip install torch gym_conservation "ray[rllib, tune]"

In [17]:
import torch
import gym
import gym_conservation
from ray import tune
# tune.register('[conservation-v6]', lambda cfg: [return env obj from here using cfg])
torch.cuda.device_count()
import gym_fishing
gym_conservation.envs import NonStationaryV6
from gym_fishing.envs import FishingCtsEnv as env1
en = gym.make("conservation-v6")
en

<OrderEnforcing<NonStationaryV6<conservation-v6>>>

In [20]:
def env_creator(env_name):
    if env_name == 'fishing-v1':
        from gym_fishing.envs import FishingCtsEnv as env
    elif env_name == "conservation-v6":
        from gym_conservation.envs import NonStationaryV6 as env
    else:
        raise NotImplementedError
    return env


In [27]:

env = env_creator('conservation-v6')
tune.register_env('conservation-v6', lambda config: env(config))


env_fish = env_creator('fishing-v1')
tune.register_env('fishing-v1', lambda config: env_fish(config))

In [28]:
# Import the RL algorithm (Trainer) we would like to use.
from ray.rllib.agents.ppo import PPOTrainer



# Configure the algorithm.
config = {
    # Environment (RLlib understands openAI gym registered strings).
    "env": "fishing-v1",
    "evaluation_interval": 10,
    # Use 2 environment workers (aka "rollout workers") that parallelly
    # collect samples from their own environment clone(s).
    "num_workers": 4,
    # Change this to "framework: torch", if you are using PyTorch.
    # Also, use "framework: tf2" for tf2.x eager execution.
    "framework": "torch",
    "num_gpus": 1,
    # Tweak the default model provided automatically by RLlib,
    # given the environment's observation- and action spaces.
    "model": {
        "fcnet_hiddens": [64, 64],
        "fcnet_activation": "relu",
    },
    # Set up a separate evaluation worker set for the
    # `trainer.evaluate()` call after training (see below).
    "evaluation_num_workers": 1,
    # Only for evaluation runs, render the env.
    "evaluation_config": {
        "render_env": True,
    }
}

# Create our RLlib Trainer.
trainer = PPOTrainer(config=config)



2022-01-26 04:47:42,426	WARNING trainer.py:1948 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Trainer.train()`. Instead, you will have to call `Trainer.evaluate()` manually in order to trigger an evaluation run.
(pid=30032) /home/cboettig/.local/share/virtualenvs/rl-ray-demo-y9hdSGSh/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
(pid=30032)   for external in metadata.entry_points().get(self.group, []):
(pid=30034) /home/cboettig/.local/share/virtualenvs/rl-ray-demo-y9hdSGSh/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
(pid=30034)   for external in metadata.entry_points().get(self.group, []):
(pid=30033) /home/cboettig/.local/share/virtualenvs/rl-ray-demo-y9hdSGSh/lib/python3.7/site-packages/ale_py/roms/utils.p

In [29]:
# Run it for n training iterations. A training iteration includes
# parallel sample collection by the environment workers as well as
# loss calculation on the collected batch and a model update.
for _ in range(3):
    print(trainer.train())

# Evaluate the trained Trainer (and render each timestep to the shell's
# output).
trainer.evaluate()


RayTaskError(TypeError): [36mray::RolloutWorker.par_iter_next()[39m (pid=30033, ip=172.18.0.5, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f02cfe9dc10>)
  File "/home/cboettig/.local/share/virtualenvs/rl-ray-demo-y9hdSGSh/lib/python3.7/site-packages/ray/util/iter.py", line 1151, in par_iter_next
    return next(self.local_it)
  File "/home/cboettig/.local/share/virtualenvs/rl-ray-demo-y9hdSGSh/lib/python3.7/site-packages/ray/rllib/evaluation/rollout_worker.py", line 381, in gen_rollouts
    yield self.sample()
  File "/home/cboettig/.local/share/virtualenvs/rl-ray-demo-y9hdSGSh/lib/python3.7/site-packages/ray/rllib/evaluation/rollout_worker.py", line 757, in sample
    batches = [self.input_reader.next()]
  File "/home/cboettig/.local/share/virtualenvs/rl-ray-demo-y9hdSGSh/lib/python3.7/site-packages/ray/rllib/evaluation/sampler.py", line 103, in next
    batches = [self.get_data()]
  File "/home/cboettig/.local/share/virtualenvs/rl-ray-demo-y9hdSGSh/lib/pyth

2022-01-26 04:48:15,380	ERROR worker.py:85 -- Unhandled error (suppress with RAY_IGNORE_UNHANDLED_ERRORS=1): ray::RolloutWorker.par_iter_next() (pid=30034, ip=172.18.0.5, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f5950b00910>)
  File "/home/cboettig/.local/share/virtualenvs/rl-ray-demo-y9hdSGSh/lib/python3.7/site-packages/ray/util/iter.py", line 1151, in par_iter_next
    return next(self.local_it)
  File "/home/cboettig/.local/share/virtualenvs/rl-ray-demo-y9hdSGSh/lib/python3.7/site-packages/ray/rllib/evaluation/rollout_worker.py", line 381, in gen_rollouts
    yield self.sample()
  File "/home/cboettig/.local/share/virtualenvs/rl-ray-demo-y9hdSGSh/lib/python3.7/site-packages/ray/rllib/evaluation/rollout_worker.py", line 757, in sample
    batches = [self.input_reader.next()]
  File "/home/cboettig/.local/share/virtualenvs/rl-ray-demo-y9hdSGSh/lib/python3.7/site-packages/ray/rllib/evaluation/sampler.py", line 103, in next
    batches = [self.get_data()]
  F